In [27]:
# going at root folder
import os
os.chdir('../')
# %pwd

## Langsmith setup

In [28]:
## Langsmith setup
from dotenv import load_dotenv

# Load variables from .env into environment
load_dotenv()

# Access them
api_key = os.getenv("LANGCHAIN_API_KEY")
endpoint = os.getenv("LANGCHAIN_ENDPOINT")
tracing = os.getenv("LANGSMITH_TRACING")

print("LangSmith endpoint:", endpoint)
print("Tracing enabled:", tracing)

LangSmith endpoint: https://api.smith.langchain.com
Tracing enabled: true


### Data Loading 

In [29]:
import mlflow
import time
import json
import dagshub
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 500
chunk_overlap = 100

#extract text from pdf files   #load all the pdf files from data folder
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    return loader.load()

In [30]:
#mlflow.set_tracking_uri("http://127.0.0.1:5000")
dagshub.init(repo_owner='264Gaurav', repo_name='medical-chatbot', mlflow=True)

# Set the experiment name
mlflow.set_experiment("RAG_ragas")

# Start an MLflow run to log this execution
with mlflow.start_run() as run:
    run_id = run.info.run_id
    print(f"MLflow Run ID: {run_id}")

    data_dir = 'data' # This is the path to your data directory
    mlflow.log_param("data_directory", data_dir)
    mlflow.log_param("pdf_loader_class", "PyPDFLoader")

    start_time = time.time()
    extracted_data = load_pdf_files(data_dir) # extracted_data will be a list of Document objects
    end_time = time.time()
    loading_duration = end_time - start_time

    mlflow.log_metric("num_documents_loaded", len(extracted_data))
    mlflow.log_metric("pdf_loading_duration_seconds", loading_duration)

    # Log the number of documents loaded - CORRECTED
    # 'extracted_data' is the list of loaded documents, so its length gives the number of documents
    num_documents = len(extracted_data)
    mlflow.log_param("num_documents", num_documents)



    print(f"MLflow Run ID: {run_id}")
    print(f"Loaded {num_documents} documents in {loading_duration:.2f} seconds.")
    print(f"MLflow run finished. View at {mlflow.get_tracking_uri()}")



Initialized MLflow to track repo "264Gaurav/medical-chatbot"

Repository 264Gaurav/medical-chatbot initialized!

MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 403 != 200. Response body: ''

### Filtering of Loaded DATA (i.e., extracted_data)

In [5]:
from typing import List
from langchain.schema import Document

##DATA cleaning and filtering
def filter_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, filter out those with new list of Document objects
    containing only 'source' and 'page' in metadata and the original page_content.

    MLflow Tracking:
    - Logs 'input_documents_count' as a metric.
    - Logs 'output_documents_count' as a metric.
    - Logs 'documents_filtered_count' as a metric.
    - Logs 'filter_function_name' as a parameter.
    """



    # Log initial state
    input_documents_count = len(docs)
    mlflow.log_metric("input_documents_count_for_filtering", input_documents_count)
    mlflow.log_param("filter_function_name", "filter_docs")

    minimal_docs: List[Document] = []

    for doc in docs:
        src = doc.metadata.get('source')
        page = doc.metadata.get('page')
        # Here, you might add more complex filtering logic if needed
        # For this specific function, it always adds the document, just with minimal metadata
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src , "page":page}
            )
        )

    output_documents_count = len(minimal_docs)
    documents_filtered_count = input_documents_count - output_documents_count # Will be 0 if all are kept

    # Log metrics after processing
    mlflow.log_metric("output_documents_count_after_filtering", output_documents_count)
    mlflow.log_metric("documents_filtered_count", documents_filtered_count)

    # Optional: Log a sample of filtered documents as an artifact
    if minimal_docs:
        sample_docs_path = "filtered_docs_sample.json"
        # Log metadata from first few documents as a sample
        sample_data = []
        for i, doc in enumerate(minimal_docs[:5]): # Log first 5 documents
            sample_data.append({
                "page_content_preview": doc.page_content[:200] + "...",
                "metadata": doc.metadata
            })
        with open(sample_docs_path, "w") as f:
            json.dump(sample_data, f, indent=4)
        mlflow.log_artifact(sample_docs_path, artifact_path="data_filtering_artifacts")
        os.remove(sample_docs_path) # Clean up local file

    return minimal_docs


minimal_docs = filter_docs(extracted_data) ## function calling here for filtering of extracted_docs

### Split the Filtered Docs (minimal_docs) into smaller chunks

In [7]:
## split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_documents(minimal_docs)


chunks = text_split(minimal_docs)
print(f"Number of text chunks: {len(chunks)}")


# Log the number of chunks created
num_chunks = len(chunks)
mlflow.log_metric("num_chunks_created", num_chunks) # Changed to metric as it's an output count

# Log text splitter parameters
mlflow.log_param("chunk_size", chunk_size)
mlflow.log_param("chunk_overlap", chunk_overlap)
mlflow.log_param("text_splitter_class", "RecursiveCharacterTextSplitter")

# Optional: Log a sample of the first few chunks as an artifact
if chunks:
    chunks_sample_path = "chunks_sample.json"
    with open(chunks_sample_path, "w") as f:
        json.dump([chunk.dict() for chunk in chunks[:5]], f, indent=4) # Log first 5 chunks
    mlflow.log_artifact(chunks_sample_path, artifact_path="data_processing_artifacts")
    os.remove(chunks_sample_path) # Clean up local file


Number of text chunks: 6600


/var/folders/38/7jcjyd5s1cd09qhrzns_rny80000gn/T/ipykernel_67318/4157002724.py:27: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump([chunk.dict() for chunk in chunks[:5]], f, indent=4) # Log first 5 chunks


### Embedding model setup

In [8]:
import time
import mlflow

# Use the LangChain Ollama embedding class
# If your LangChain installation exposes it under a different module, see the note below.
from langchain_ollama.embeddings import OllamaEmbeddings


def download_embeddings(
    model_name: str = "nomic-embed-text:latest",
    base_url: str = "http://localhost:11434",
):
    """
    Connects to the locally-running Ollama instance and returns an OllamaEmbeddings object
    using the specified embedding model (nomic-embed-text:latest by default).
    """

    # Log embedding model parameters
    mlflow.log_param("embedding_model_name", model_name)
    mlflow.log_param("embedding_provider", "Ollama")
    mlflow.log_param("embedding_class", "OllamaEmbeddings")
    mlflow.log_param("ollama_base_url", base_url)

    start_time = time.time()
    # Create the LangChain wrapper for Ollama embeddings
    embeddings = OllamaEmbeddings(model=model_name, base_url=base_url)
    end_time = time.time()
    loading_duration = end_time - start_time

    # Log metrics
    mlflow.log_metric("embedding_loading_duration_seconds", loading_duration)

    print(f"Connected to Ollama embedding model '{model_name}' at {base_url} (init {loading_duration:.2f}s).")

    return embeddings



embedding = download_embeddings()
## example: embedding.embed_documents(["hello world", "another doc"])


Connected to Ollama embedding model 'nomic-embed-text:latest' at http://localhost:11434 (init 0.01s).


In [9]:
vector = embedding.embed_query("Hello to medical chatbot.")

vector_dim = len(vector);
print(f"Vector length: {vector_dim}")  # Check the length of the vector  ## Dimentions of the vector

mlflow.log_param("vector_embedding_size", vector_dim)


Vector length: 768


768

### Vector DB setup

In [10]:
## Pinecone is the leading vector database for building accurate and performant AI applications
from pinecone import Pinecone


PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
pinecone_api = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api)
pc

In [11]:
from pinecone import Pinecone, ServerlessSpec


# Log stage name
mlflow.log_param("stage", "pinecone_index_setup")

index_name = 'medical-chatbot'
dimension = vector_dim  # Dimension of the embeddings (should match your embedding model output)
metric = 'cosine' # Similarity metric

# Log Pinecone index parameters
mlflow.log_param("pinecone_index_name", index_name)
mlflow.log_param("pinecone_index_dimension", dimension)
mlflow.log_param("pinecone_index_metric", metric)

# Define the serverless spec parameters
cloud_provider = "aws"
region = "us-east-1"
mlflow.log_param("pinecone_cloud_provider", cloud_provider)
mlflow.log_param("pinecone_region", region)


# Check if index exists and create if not
start_check_time = time.time()
index_exists = pc.has_index(index_name)
end_check_time = time.time()
check_duration = end_check_time - start_check_time

mlflow.log_metric("pinecone_index_exists_check_duration_seconds", check_duration)
mlflow.log_param("pinecone_index_existed_before_run", index_exists)

if not index_exists:
    mlflow.log_param("pinecone_index_action", "created_new_index")
    print(f"Pinecone index '{index_name}' does not exist. Creating...")
    start_create_time = time.time()
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(cloud=cloud_provider, region=region)
    )
    end_create_time = time.time()
    creation_duration = end_create_time - start_create_time
    mlflow.log_metric("pinecone_index_creation_duration_seconds", creation_duration)
    print(f"Pinecone index '{index_name}' created in {creation_duration:.2f} seconds.")
else:
    mlflow.log_param("pinecone_index_action", "connected_to_existing_index")
    print(f"Pinecone index '{index_name}' already exists. Connecting...")
    mlflow.log_metric("pinecone_index_creation_duration_seconds", 0) # Log 0 if not created

# Connect to the index
start_connect_time = time.time()
index = pc.Index(index_name)
end_connect_time = time.time()
connect_duration = end_connect_time - start_connect_time

mlflow.log_metric("pinecone_index_connection_duration_seconds", connect_duration)
print(f"Connected to Pinecone index '{index_name}' in {connect_duration:.4f} seconds.")

# Optional: Log some basic info about the connected index (e.g., number of vectors)
# This might require querying the index, which adds time, so consider if needed for every run.
try:
    index_info = index.describe_index_stats()
    mlflow.log_metric("pinecone_total_vector_count", index_info.dimension) # This gives dimension, not vector count directly
    # To get actual vector count:
    if index_info.namespaces:
        total_vectors_in_index = sum(ns.vector_count for ns_name, ns in index_info.namespaces.items())
        mlflow.log_metric("pinecone_total_vectors_in_index", total_vectors_in_index)
        mlflow.log_param("pinecone_namespaces", list(index_info.namespaces.keys()))
except Exception as e:
    print(f"Could not get Pinecone index stats: {e}")
    mlflow.log_param("pinecone_index_stats_error", str(e))


Pinecone index 'medical-chatbot' does not exist. Creating...
Pinecone index 'medical-chatbot' created in 11.97 seconds.
Connected to Pinecone index 'medical-chatbot' in 0.0265 seconds.


/opt/anaconda3/envs/medical/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Injection of filtered and chunked data (i.e., chunks) into Pinecone vector DB

In [14]:
from langchain_pinecone import PineconeVectorStore

def upsert_to_pinecone(chunks, embedding, index_name, batch_size: int = 50):
    """
    Upserts documents to a Pinecone index in safe batches (<4MB per request).
    Logs metadata and timing with MLflow.
    """
    # Log input parameters
    mlflow.log_param("vector_store_type", "PineconeVectorStore")
    mlflow.log_param("index_name_for_upsertion", index_name)
    mlflow.log_param("num_text_chunks_for_upsertion", len(chunks))
    mlflow.log_param("batch_size for pinecone data injection", batch_size)
    if hasattr(embedding, "model"):
        mlflow.log_param("embedding_model_used_for_upsertion", getattr(embedding, "model", "unknown"))

    print(f"Starting batched upsertion to Pinecone index '{index_name}' with {len(chunks)} chunks...")

    start_time = time.time()

    # Upsert in batches to avoid Pinecone 4MB limit
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i : i + batch_size]
        PineconeVectorStore.from_documents(
            documents=batch,
            embedding=embedding,
            index_name=index_name
        )
        print(f"  Upserted batch {i // batch_size + 1} ({len(batch)} chunks).")

    end_time = time.time()
    upsertion_duration = end_time - start_time

    # Log metrics
    mlflow.log_metric("pinecone_upsertion_duration_seconds", upsertion_duration)

    print(f"✅ Upsertion to Pinecone completed in {upsertion_duration:.2f} seconds.")



# upsert_to_pinecone(chunks=chunks,  embedding=embedding, index_name=index_name, batch_size= 500)  ## uncomment only when you want to insert the data in pinecode DB (i.e., if data in DB not injected before)



### Retriever

In [15]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

# Define parameters for retrieval
retrieval_search_type = "similarity"
retrieval_k = 5
test_query = "What is Cancer?"

# 1. Log Retrieval Parameters
mlflow.log_param("retriever_search_type", retrieval_search_type)
mlflow.log_param("retriever_k_value", retrieval_k)
mlflow.log_param("retrieval_query", test_query) # Log the specific query used

# Configure the retriever
retriever = docsearch.as_retriever(search_type=retrieval_search_type, search_kwargs={"k": retrieval_k})

# 2. Measure and Log Retrieval Time on Test Query
start_time = time.time()
retrieved_docs = retriever.invoke(test_query) ## Retriever invoked
end_time = time.time()
retrieval_duration = end_time - start_time
mlflow.log_metric("retrieval_duration_seconds", retrieval_duration)

# 3. Log Retrieved Document Count
num_retrieved = len(retrieved_docs)
mlflow.log_metric("num_retrieved_documents", num_retrieved)


# 4. Log a Sample of Retrieved Documents as an Artifact
if retrieved_docs:
    sample_docs = []
    # Log details of the first 3 documents as a sample
    for i, doc in enumerate(retrieved_docs[:3]):
        sample_docs.append({
            "index": i + 1,
            "source": doc.metadata.get('source', 'N/A'),
            "page": doc.metadata.get('page', 'N/A'),
            "content_preview": doc.page_content[:200] + "..." # Log first 200 chars
        })

    temp_file_path = "retrieved_docs_sample.json"
    with open(temp_file_path, "w") as f:
        json.dump(sample_docs, f, indent=4)

    mlflow.log_artifact(temp_file_path, artifact_path="retrieval_output_samples")
    os.remove(temp_file_path) # Clean up the local temporary file

print(f"Retrieved {num_retrieved} documents for '{test_query}' in {retrieval_duration:.4f} seconds.")


Retrieved 5 documents for 'What is Cancer?' in 1.7099 seconds.


## LLM Model setup 

In [16]:
from langchain_community.llms import Ollama  # wrapper for local Ollama models


# Define LLM parameters
llm_model_name = "llama3.1:latest"
llm_temperature = 0
llm_timeout = None
llm_max_retries = 2

# Log LLM parameters
mlflow.log_param("llm_provider", "Ollama")
mlflow.log_param("llm_class", "Ollama")
mlflow.log_param("llm_model_name", llm_model_name)
mlflow.log_param("llm_temperature", llm_temperature)
mlflow.log_param("llm_timeout", llm_timeout if llm_timeout is not None else "None")
mlflow.log_param("llm_max_retries", llm_max_retries)


print(f"Initializing Ollama model: {llm_model_name}...")
start_time = time.time()
chatModel = Ollama(
    model=llm_model_name,
    temperature=llm_temperature,
    # Ollama wrapper doesn’t use `max_tokens` or `timeout` exactly the same way as Google GenAI,
    # but you can still pass client kwargs if needed.
)
end_time = time.time()
initialization_duration = end_time - start_time

# Log metrics
mlflow.log_metric("llm_initialization_duration_seconds", initialization_duration)

print(f"Ollama model '{llm_model_name}' initialized in {initialization_duration:.4f} seconds.")


Initializing Ollama model: llama3.1:latest...
Ollama model 'llama3.1:latest' initialized in 0.0002 seconds.


/var/folders/38/7jcjyd5s1cd09qhrzns_rny80000gn/T/ipykernel_67318/632503613.py:21: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  chatModel = Ollama(


In [17]:
ans = chatModel.invoke("who is MS Dhoni")
ans

"MS Dhoni, also known as Mahendra Singh Dhoni, is a former Indian international cricketer who played for the Indian national team from 2004 to 2019. He was one of the most successful and popular cricketers in the world during his playing career.\n\nHere are some key facts about MS Dhoni:\n\n**Early Life**\n\nDhoni was born on July 7, 1981, in Ranchi, Jharkhand (then part of Bihar), India. His father, Pan Singh Dhoni, was a railway employee, and his mother, Devki Devi, was a homemaker.\n\n**Cricket Career**\n\nDhoni started playing cricket at the age of eight and quickly rose through the ranks to become one of the most successful wicket-keepers in Indian cricket history. He made his international debut in 2004 against Bangladesh and went on to play for India in all formats of the game (Test, ODI, T20).\n\n**Achievements**\n\nDhoni's achievements are numerous:\n\n1. **Captaincy**: Dhoni was a successful captain of the Indian team, leading them to several victories, including the 2011 Cri

In [18]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

## for ready-made RAG prompts
from langchain import hub
# Pull a pre-made RAG prompt from LangChain Hub
prompt = hub.pull("rlm/rag-prompt")



## System Prompt defining
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Keep the answer concise and understandable."
    "\n\n"
    "{context}"
)

# Create the ChatPromptTemplate
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )


mlflow.log_param("system prompt for llm :", system_prompt)
print(prompt)
print("\n\n Prompt template defined and logged to MLflow.")

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


 Prompt template defined and logged to MLflow.


### Defining RAG chain

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Helper function to format retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the full RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)

In [24]:
# Ask a question using the RAG chain
response = rag_chain.invoke("What dose of vitamin-D should we take?")
print(response)

The usual dosage of vitamin D for adults is 400 mg, taken two to three times a day with meals. For children under six years old, the usual dosage is 50 mg per day, divided into several small doses. The dose may be different for different people, so it's best to check with a physician or pharmacist for the correct dosage.


## RAGAS Setup: 1.TestSet Generation

In [21]:
from ragas.testset.graph import Node

sample_nodes = [Node(
    properties={"page_content": "Einstein's theory of relativity revolutionized our understanding of space and time. It introduced the concept that time is not absolute but can change depending on the observer's frame of reference."}
),Node(
    properties={"page_content": "Time dilation occurs when an object moves close to the speed of light, causing time to pass slower relative to a stationary observer. This phenomenon is a key prediction of Einstein's special theory of relativity."}
)]
sample_nodes

[Node(id: 2bdf86, type: NodeType.UNKNOWN, properties: ['page_content']),
 Node(id: 77712f, type: NodeType.UNKNOWN, properties: ['page_content'])]